ACISpy provides several classes for plotting various quantities. These plots can be 
modified and saved to disk, or used in an interactive session. To make plots appear in an
interactive [IPython](http://ipython.org/) session, do one of the following:

* In an IPython console: start as `ipython --matplotlib`
* In an IPython Qt console or notebook: start the first cell with `%matplotlib inline`

This documentation page is actually a runnable IPython notebook. A link to the raw notebook can be found at the bottom of the page. 

For the example plots we'll show, we'll use this `Dataset`:

In [ ]:
# DON'T USE THIS LINE IF YOU ARE RUNNING IN THE IPYTHON COMMAND-LINE CONSOLE
%matplotlib inline

In [ ]:
import acispy
msids = ["1dpamzt", "1deamzt", "1dp28avo","aopcadmd"]
ds = acispy.ArchiveData("2016:318", "2016:338", msid_keys=msids, interpolate_msids=True)

## Creating Plots of Data vs. Time

### DatePlot

The `DatePlot` object can be used to make a single-panel plot of one
or more quantities versus the date and time. 

Plot of one MSID vs. time:


In [ ]:
dp1 = acispy.DatePlot(ds, "1dpamzt")

Plot of two MSIDs together vs. time:

In [ ]:
dp2 = acispy.DatePlot(ds, ["1dpamzt", "1deamzt"])

Plot of an MSID on the left y-axis and a state on the right y-axis:

In [ ]:
dp3 = acispy.DatePlot(ds, "1dpamzt", field2="pitch") 

A number of options can be used to modify the `DatePlot` when creating it. For example, the width and color of the lines can be changed:

In [ ]:
dp4 = acispy.DatePlot(ds, "1dpamzt", field2="pitch", lw=2, color="green", color2="purple")  

Some MSIDs do not correspond to particular numbers but have "state" values. These can be plotted as well:

In [ ]:
dp = acispy.DatePlot(ds, "1dpamzt", field2="aopcadmd")
dp.tight_layout()

### CustomDatePlot

One may want to make a plot of data vs. time that isn't restricted to the quantities in a `Dataset`. For this, ACISpy provides a `CustomDatePlot` class which can take an array of date/time strings and a corresponding array of quantities.

In [ ]:
# Ratio of 1DEAMZT to 1DPAMZT
times = ds.dates("1deamzt")
tratio = ds["1deamzt"]/ds["1dpamzt"]
dp5 = acispy.CustomDatePlot(times, tratio)
dp5.set_ylabel(r"$\mathrm{T_{DEA}/T_{DPA}}$")

## Creating Multi-Panel Plots

The `MultiDatePlot` object can be used to make a multiple-panel plot of multiple quantities versus the date and time. 

By default the panels are stacked vertically:

In [ ]:
mdp1 = acispy.MultiDatePlot(ds, ["pitch", "1deamzt", "ccd_count"], lw=2, fontsize=17)

But by using the `subplots` keyword argument, the panels can be arranged in a `(n_plot_x, n_plot_y)` fashion:

In [ ]:
panels = ["pitch", "1deamzt", "ccd_count", "1dpamzt"]
mdp2 = acispy.MultiDatePlot(ds, panels, subplots=(2,2))

By nesting lists, it's also possible to make subplots which have more than one quantity in a panel:

In [ ]:
# Note the first item in the list is another list
fields = [["1deamzt", "1dpamzt"], "ccd_count"]                                                  

In [ ]:
mdp3 = acispy.MultiDatePlot(ds, fields, lw=2, fontsize=17)

You can access each individual `DatePlot` panel in the `MultiDatePlot` in key-like fashion. For panels with more than one field plotted, the key for that panel is the specification for the first field:

In [ ]:
print(mdp1["ccd_count"])
print(mdp3["1deamzt"]) # This one has "1dpamzt" also

The above is not very useful by itself, but will come in handy later when we want to make modifications to individual panels, as we'll show below. 

## Creating Phase Plots

A `PhasePlot` shows one quantity plotted versus another. This can behelpful when trying to determine the behavior of one MSID versus another, or the dependence of an MSID on a particular commanded state. 

There is an important caveat about `PhasePlot`s: it is required that the data are interpolated to a common set of times. This requires that you have set `interpolate_msids=True` in the `ArchiveData` call, or that you have mapped a state to a MSID. 

There are two kinds of `PhasePlot`s in ACISpy: a `PhaseScatterPlot` and a `PhaseHistogramPlot`, which both plot one quantity vs. another but the former shows them as scattered points and the latter bins them up into a 2D histogram.

A histogram plot of one MSID vs. another:

In [ ]:
xbins = 50 # These can be integers or arrays specifying the bin edges
ybins = 50 # These can be integers or arrays specifying the bin edges
pp1 = acispy.PhaseHistogramPlot(ds, "1dpamzt", "1deamzt", xbins, ybins, scale='log')

A scatter plot of a MSID vs. a state:

In [ ]:
# Must map the state to the MSID times we want to map them to
ds.map_state_to_msid("pitch", "1deamzt")
pp2 = acispy.PhaseScatterPlot(ds, ("msids", "pitch"), "1deamzt")

A plot of one state vs. another (no interpolation required in this case):

In [ ]:
pp3 = acispy.PhaseScatterPlot(ds, ("states", "pitch"), "roll")

`PhaseScatterPlot`s can also have their data points colored by a third quantity. To do this, we simply specify a third field in the argument `c_field`. Like the other fields, it must be interpolated to the same set of times. The `cmap` argument can change the colormap.

In [ ]:
pp4 = acispy.PhaseScatterPlot(ds, "1dpamzt", "1deamzt", c_field=('msids', 'pitch'), cmap='hsv')

## Plot Modifications

The various plotting classes have methods to modify the plots after creating them. These include methods to control the limits of the plots, change plot labels, add titles, text, legends, lines, and grids, and save plots to disk. 

### Changing Plot Limits

For `DatePlot` and `MultiDatePlot`, the date/time limits on the x-axis can be set using `DatePlot.set_xlim`. For example, the single plot of 1DPAMZT above can be rescaled:

In [ ]:
dp1.set_xlim("2016:320", "2016:328")
dp1

For `DatePlot` objects, `set_ylim()` and `set_ylim2()` can be used to control the limits of the
left and right y-axes of the plot, respectively:

In [ ]:
dp3.set_ylim(10, 35)
dp3.set_ylim2(60, 140)
dp3

Since the individual panels of each `MultiDatePlot` are `DatePlot` instances, these methods work on the individual panels as well (note here the limits of the bottom panel change):

In [ ]:
mdp1["ccd_count"].set_ylim(0, 7)
mdp1

### Changing Plot Labels

`set_ylabel()` and `set_ylabel2()` can be used to control the labels of the left and right y-axes of a `DatePlot`, respectively:

In [ ]:
dp3.set_ylabel("DPA Temperature")
dp3.set_ylabel2("Pitch Angle")
dp3

`PhaseScatterPlot` and `PhaseHistogramPlot` have similar methods for setting the labels on the x and y-axes, `set_xlabel()` and `set_ylabel()`:

### Adding Vertical and Horizontal Lines to a Plot

Vertical and horizontal lines may be added to any of the plot types using the `add_hline()` and `add_vline()` methods. The appearance of the lines can be controlled. For example, we'll add a vertical dashed brown line on plot `dp1` at midnight on day 16 of the year 2015, with a line width of 3. 

In [ ]:
dp1.add_vline("2016:325:00:00:00.000", lw=3, ls='dashed', color='brown')
dp1

Next, we'll add a green horizontal dash-dot line at 25$^\circ$C:

In [ ]:
dp1.add_hline(25, lw=3, ls='dashdot', color='green')
dp1

For a `DatePlot` with both left and right y-axes, horizonal lines can be added for both scales (use `add_hline2()` for the right y-axis):

In [ ]:
dp3.add_hline(20, lw=2, ls='solid', color='green')
dp3.add_hline2(110, lw=2, ls='dotted', color='brown')
dp3

Adding a vertical line to a `MultiDatePlot` adds it to all panels, whereas to add a horizontal line to a panel you must add it to the individual plot:

In [ ]:
mdp1.add_vline("2016:330:12:45:56.031", color='purple', lw=3, ls='dashed')
mdp1["ccd_count"].add_hline(5, color='green', lw=2)
mdp1

### Adding a Title to a Plot

The `set_title()` method for any of the plot types can be used to add a title to the top of the plot:

In [ ]:
mdp1.set_title("Three Plots", fontsize=20, loc='left') # "loc" sets the horizontal location of the title
mdp1

In [ ]:
pp1.set_title("Temperature vs. Temperature", fontsize=18)
pp1

### Customizing a Legend on a `DatePlot`

A `DatePlot` with multiple lines plotted on the left y-axis has a legend. This legend can be customized, by moving its location or changing the font size, using `set_legend()`:

In [ ]:
dp2.set_legend(loc="lower left", fontsize=20) # loc sets location
dp2

If you want to modify one of the labels in the legend, use the `set_field_label()` method:

In [ ]:
dp2.set_field_label("1deamzt", "DEA Temperature")
dp2

### Adding Grid Lines to a Plot

For any of the plot types, call the `set_grid()` to method to turn grid lines on 
and off on the plot:

In [ ]:
dp3.set_grid(True)
dp3

### Adding Text to Plots

Text can be added to a `DatePlot` or `PhasePlot` by calling the `add_text()` method:

In [ ]:
dp1.add_text("2016:321:00:00:00", 8, "The DPA gets hotter.")
dp1

In [ ]:
pp1.add_text(30, 10, "No data down here!", color='red', fontsize=17, rotation='45')
pp1

### Saving Plots to Disk

Finally, for any of the plot types, call `savefig()` to save the figure:

In [ ]:
pp1.savefig("phase_plot.png")